# Ping miner pools with stratum-ping
- https://github.com/2miners/stratum-ping
- https://www.reddit.com/r/EtherMining/comments/o7l0qv/stratum_ping_times_for_your_location/

Used to check if their is problem with specific pool or port.
Check extra if firewall is not correct.

When their is proxy problem test
- https://stratumproxy.io/

In [95]:
# Download stratum-ping
# !wget https://github.com/2miners/stratum-ping/releases/download/v1.0.0/stratum-ping-linux-x64.tar.gz
# !tar -xf stratum-ping-linux-x64.tar.gz

In [96]:
import subprocess
import re

In [97]:
# check startum-ping is downloaded
!./stratum-ping -h

Usage of ./stratum-ping:
  -6	use ipv6
  -c int
    	stop after <count> replies (default 5)
  -p string
    	pass (default "x")
  -t string
    	stratum type: stratum1, stratum2 (default "stratum2")
  -tls
    	use TLS
  -u string
    	login (default "0x63a14c53f676f34847b5e6179c4f5f5a07f0b1ed")


In [98]:
# run one test
!./stratum-ping -c 2 stratum-ethw.antpool.com:443

PING stratum stratum-ethw.antpool.com (172.65.231.105) port 443
stratum-ethw.antpool.com (172.65.231.105): seq=0, time=584.9129ms
stratum-ethw.antpool.com (172.65.231.105): seq=1, time=594.6175ms

--- stratum-ethw.antpool.com ping statistics ---
2 packets transmitted, 2 received, 0% packet loss, time 3.2397408s
min/avg/max = 584.9129ms, 589.7652ms, 594.6175ms


## Servers

In [99]:
rvn_servers = [
    ["Cruxpool", "Europe", "stratum+tcp://eu.cruxpool.com:9999"],
    ["HeroMiners", "Global", "stratum+tcp://ravencoin.herominers.com:10240"],
    ["HeroMiners", "Global", "stratum+tcp://fi.ravencoin.herominers.com:1140"],
    ["BSOD.PW", "Europe", "stratum+tcp://eu.bsod.pw:2640"],
    ["MinerMore", "Europe", "stratum+tcp://eu.rvn.minermore.com:4501"],
    ["Suprnova", "Global", "stratum+tcp://rvn.suprnova.cc:8888"],
    ["Kryptex", "Global", "stratum+tcp://rvn.kryptex.network:7777"],
    ["Kryptex", "Global", "stratum+tcp://rvn.kryptex.network:8888"],
    ["2miners.com", "Global", "stratum+tcp://rvn.2miners.com:6060"],
    ["RavenMiner", "Global", "stratum+tcp://stratum.ravenminer.com:3838"],
    ["Nanopool", "Europe 1", "stratum+tcp://rvn-eu1.nanopool.org:12222"],
    ["Nanopool", "Europe 1 (SSL)", "stratum+ssl://rvn-eu1.nanopool.org:12433"],
    ["K1Pool", "Europe", "stratum+tcp://eu.rvn.k1pool.com:7861"]
]


In [100]:
eth_servers = [
    ["2Miners", "Global", "stratum+tcp://eth.2miners.com:2020"],
    ["HelloMiner", "EU1", "stratum+tcp://eth-eu1.hellominer.com:1100"],
    ["Unmineable", "Global", "stratum+tcp://ethash.unmineable.com:3333"],
    ["F2Pool", "Global", "stratum+tcp://etc.f2pool.com:8118"],
    ["F2Pool", "Global", "stratum+tcp://eth.f2pool.com:6688"]
]


In [101]:
test_servers = [
    # Adding AntPool servers
    ["AntPool", "Global", "stratum+tcp://stratum.antpool.com:25"],
    ["AntPool", "Global", "stratum+tcp://stratum.antpool.com:3333"],
    ["AntPool", "Global", "stratum+tcp://stratum.antpool.com:443"],
    ["AntPool", "Global", "stratum+tcp://stratum.antpool.com:445"],
    # Adding Ravencoin server on AntPool
    ["AntPool", "RVN", "stratum+tcp://rvn.antpool.com:8003"],
    # Adding Kaspa server on AntPool
    ["AntPool", "KAS", "stratum+tcp://kas.antpool.com:445"],
    # Adding EthereumWorld servers on AntPool
    ["AntPool", "ETHW", "stratum+tcp://stratum-ethw.antpool.com:25"],
    ["AntPool", "ETHW", "stratum+tcp://stratum-ethw.antpool.com:443"],
    # Adding Poolin servers for EthereumWorld
    ["Poolin", "ETHW", "stratum+tcp://ethw.ss.poolin.one:443"],
    ["Poolin", "ETHW", "stratum+tcp://ethw.ss.poolin.one:1883"],
    ["Poolin", "ETHW", "stratum+tcp://ethw.ss.poolin.one:700"],
    ["Xmrig", "Global", "stratum+tcp://donate.ssl.xmrig.com:443"]
]

In [102]:
# combine all servers
all_servers = test_servers + rvn_servers + eth_servers

## Check all servers

In [103]:
def successful_transmission(text):
    # Regex to find the number of packets received
    pattern = r", (\d+) received,"
    match = re.search(pattern, text)
    if match:
        return int(match.group(1))
    else:
        return 0

def port_number(text):
    # Regex to find the number of packets received
    pattern = r":(\d+)"
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return ''

def run_stratum_ping(server):
    # The command to run, assuming stratum-ping is in the current working directory or in the PATH
    command = ['./stratum-ping', '-c', '1', server]

    # Execute the command
    try:
        # If you want to capture the output
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # # Print the standard output
        # print(result.stdout)

        # # Check if there were any errors
        # if result.stderr:
        #     print("Errors:\n", result.stderr)
    except subprocess.CalledProcessError as e:
        print("An error occurred while trying to execute stratum-ping:", str(e))
    except FileNotFoundError:
        print("stratum-ping executable not found. Ensure it's in the current working directory or its path is correctly set.")

    return [successful_transmission(result.stdout), server, port_number(server), result.stdout]

def check_stratum_ping(servers):
    for i, server in enumerate(servers):
        print(f'\n\n---------- START {i} -----------------')
        res = run_stratum_ping(server)
        print(res)

def check_all_servers(servers):
    results = []
    # strip protocol from adress
    server_addresses = [server[2].replace('stratum+ssl://', '').replace('stratum+tcp://', '') for server in servers]
    for server in server_addresses:
        res = run_stratum_ping(server)
        print(res)
        results.append(res)

    return results

In [104]:
# sample test
# res = check_stratum_ping(['donate.ssl.xmrig.com:443'])
check_stratum_ping(['eth.f2pool.com:6688', 'rvn-asia1.nanopool.org:12433'])



---------- START 0 -----------------
[1, 'eth.f2pool.com:6688', '6688', 'PING stratum eth.f2pool.com (172.65.212.50) port 6688\neth.f2pool.com (172.65.212.50): seq=0, time=87.6025ms\n\n--- eth.f2pool.com ping statistics ---\n1 packets transmitted, 1 received, 0% packet loss, time 1.1380375s\nmin/avg/max = 87.6025ms, 87.6025ms, 87.6025ms\n']


---------- START 1 -----------------
[0, 'rvn-asia1.nanopool.org:12433', '12433', 'PING stratum rvn-asia1.nanopool.org (172.104.165.191) port 12433\nrvn-asia1.nanopool.org (172.104.165.191): seq=0, EOF\n\n--- rvn-asia1.nanopool.org ping statistics ---\n1 packets transmitted, 0 received, 100% packet loss, time 1.6415962s\n']


In [105]:
res = check_all_servers(all_servers)
# res = check_all_servers(eth_servers)
# res = check_all_servers(rvn_servers)
# res = check_all_servers(test_servers)

[0, 'stratum.antpool.com:25', '25', 'PING stratum stratum.antpool.com (172.65.209.66) port 25\nstratum.antpool.com (172.65.209.66): seq=0, dial tcp4 172.65.209.66:25: connect: connection timed out\n\n--- stratum.antpool.com ping statistics ---\n1 packets transmitted, 0 received, 100% packet loss, time 2m11.4566548s\n']
[0, 'stratum.antpool.com:3333', '3333', 'PING stratum stratum.antpool.com (172.65.209.66) port 3333\nstratum.antpool.com (172.65.209.66): seq=0, read tcp4 172.26.173.233:56722->172.65.209.66:3333: i/o timeout\n\n--- stratum.antpool.com ping statistics ---\n1 packets transmitted, 0 received, 100% packet loss, time 11.028427s\n']
[1, 'stratum.antpool.com:443', '443', 'PING stratum stratum.antpool.com (172.65.209.66) port 443\nstratum.antpool.com (172.65.209.66): seq=0, time=78.7785ms\n\n--- stratum.antpool.com ping statistics ---\n1 packets transmitted, 1 received, 0% packet loss, time 1.112345s\nmin/avg/max = 78.7785ms, 78.7785ms, 78.7785ms\n']
[0, 'stratum.antpool.com:44

## Result

In [106]:
# sort list on 1 & 3 columns
sorted_list = sorted(res, key=lambda x: (x[0], x[2]), reverse=True)
# display the 3 first elememts for all items
[item[:3] for item in sorted_list]

[[1, 'eu.cruxpool.com:9999', '9999'],
 [1, 'rvn.suprnova.cc:8888', '8888'],
 [1, 'etc.f2pool.com:8118', '8118'],
 [1, 'rvn.antpool.com:8003', '8003'],
 [1, 'eu.rvn.k1pool.com:7861', '7861'],
 [1, 'rvn.kryptex.network:7777', '7777'],
 [1, 'ethw.ss.poolin.one:700', '700'],
 [1, 'eth.f2pool.com:6688', '6688'],
 [1, 'rvn.2miners.com:6060', '6060'],
 [1, 'eu.rvn.minermore.com:4501', '4501'],
 [1, 'stratum.antpool.com:443', '443'],
 [1, 'stratum-ethw.antpool.com:443', '443'],
 [1, 'ethw.ss.poolin.one:443', '443'],
 [1, 'stratum.ravenminer.com:3838', '3838'],
 [1, 'ethash.unmineable.com:3333', '3333'],
 [1, 'eth.2miners.com:2020', '2020'],
 [1, 'ethw.ss.poolin.one:1883', '1883'],
 [1, 'rvn-eu1.nanopool.org:12222', '12222'],
 [1, 'fi.ravencoin.herominers.com:1140', '1140'],
 [1, 'ravencoin.herominers.com:10240', '10240'],
 [0, 'rvn.kryptex.network:8888', '8888'],
 [0, 'stratum.antpool.com:445', '445'],
 [0, 'kas.antpool.com:445', '445'],
 [0, 'donate.ssl.xmrig.com:443', '443'],
 [0, 'stratum.a